In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
# City codification aggregation by latitude and longitude
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
import os
os.chdir('..')

from config import weather_api_key
from config import g_key

In [2]:
# Create a set of random latitude and longitude combinations. Sample of 2,000
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Get the nearest city using the citipy module.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

738

In [4]:
# cities

In [5]:
# Perform an API call with the OpenWeatherMap.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# url

In [6]:
# Get the JSON text of the 'Get' request - See the limit of request
city_weather = requests.get(url)
city_weather.json()

{'cod': '400', 'message': 'Nothing to geocode'}

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Part 1 END
# Loop through all the cities in our list.
for i in range(len(cities)):
    
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]
    
    # Loop through all the cities in the list.
for i, city in enumerate(cities):
#END part 2

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# END part 3
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_describe = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_describe})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

rd 42 of Set 20 | arlit
Processing Record 43 of Set 20 | falam
Processing Record 44 of Set 20 | vostok
Processing Record 45 of Set 20 | caceres
Processing Record 46 of Set 20 | vardo
Processing Record 47 of Set 20 | les cayes
Processing Record 48 of Set 20 | tiznit
Processing Record 49 of Set 20 | tucumcari
Processing Record 50 of Set 20 | ati
Processing Record 1 of Set 21 | xining
Processing Record 2 of Set 21 | marsh harbour
Processing Record 3 of Set 21 | malmyzh
Processing Record 4 of Set 21 | siilinjarvi
Processing Record 5 of Set 21 | sur
Processing Record 6 of Set 21 | ostrovnoy
Processing Record 7 of Set 21 | shimoda
Processing Record 8 of Set 21 | proletarsk
Processing Record 9 of Set 21 | kamyshin
Processing Record 10 of Set 21 | amderma
City not found. Skipping...
Processing Record 11 of Set 21 | samusu
City not found. Skipping...
Processing Record 12 of Set 21 | olavarria
Processing Record 13 of Set 21 | borba
Processing Record 14 of Set 21 | alexandria
Processing Record 15

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Kaitangata,-46.2817,169.8464,48.99,79,0,4.00,NZ,clear sky
1,Mahebourg,-20.4081,57.7000,80.60,74,40,11.50,MU,scattered clouds
2,Seoul,37.5683,126.9778,32.00,93,100,3.44,KR,mist
3,Aklavik,68.2191,-135.0107,-25.60,68,20,12.66,CA,snow
4,Ushuaia,-54.8000,-68.3000,48.20,87,90,13.80,AR,drizzle
5,Awjilah,29.1081,21.2869,64.44,26,0,11.52,LY,clear sky
6,Nikolskoye,59.7035,30.7861,7.00,85,90,6.71,RU,light snow
7,Namibe,-15.1961,12.1522,75.07,72,37,6.33,AO,scattered clouds
8,Butaritari,3.0707,172.7902,81.23,78,63,14.47,KI,broken clouds
9,Fukue,32.6881,128.8419,52.97,64,100,8.41,JP,overcast clouds


In [9]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")